In [1]:
#!pip3 install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 215kB 28.1MB/s 
     |████████████████████████████████| 8.2MB 48.2MB/s 
     |████████████████████████████████| 2.3MB 34.0MB/s 
     |████████████████████████████████| 1.2MB 50.3MB/s 
     |████████████████████████████████| 1.8MB 49.4MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 225kB 51.1MB/s 
     |████████████████████████████████| 3.3MB 36.3MB/s 
     |████████████████████████████████| 122kB 60.2MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 4.2MB 47.8MB/s 
     |████████████████████████████████| 174kB 53.9MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 112kB 59.1MB/s 
     |████████████████████████████████| 901kB 45.5MB/s 
     |████████████████████████████████| 133kB 57.9MB/s 
     |████████████████████████████████| 102kB 15.2MB/

In [2]:
#!pip3 install sentencepiece

In [1]:
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm, notebook
from tqdm import tqdm
import textfeatures as tf

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:

df = pd.read_csv('twitter_racism_parsed_dataset.csv',encoding = 'iso-8859-1')
df.head()

,index,id,Text,Annotation,oh_label
0,5.767493e+17,5.767493e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.408905e+17,5.408905e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0
2,5.678433e+17,5.678433e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.766462e+17,5.766462e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.713492e+17,5.713492e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [3]:
df.columns

Index(['index', 'id', 'Text', 'Annotation', 'oh_label'], dtype='object')

In [4]:
df = df.iloc[:, 1:]

In [5]:
df.head()

,id,Text,Annotation,oh_label
0,5.767493e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.408905e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0
2,5.678433e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.766462e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.713492e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [6]:
df = df[['Text', 'oh_label']]

In [7]:
df.isnull().sum()

Text        0
oh_label    0
dtype: int64

In [8]:
df['Text'] = df['Text'].fillna('None')
df['oh_label'] = df['oh_label'].fillna(0)

In [9]:
# Removed the stopword
tf.stopwords(df,"Text","stopword")
df[["Text","stopword"]].head()

,Text,stopword
0,@AAlwuhaib1977 Muslim mob violence against Hin...,"[against, in, in]"
1,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,[]
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,"[is, the, of, is, a, of, and, it, be]"
3,"Finally I'm all caught up, and that sudden dea...","[all, and, that, off, it's, be]"
4,@carolinesinders @herecomesfran *hugs*,[]


In [10]:
tf.clean(df,"Text","CText")
df[["Text","CText"]].head()

,Text,CText
0,@AAlwuhaib1977 Muslim mob violence against Hin...,aalwuhaib muslim violence hindus bangladesh co...
1,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,tem_nightmre
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,jncatron isra_jourisra ampalestine islamophobi...
3,"Finally I'm all caught up, and that sudden dea...",finally caught sudden death cook looks like go...
4,@carolinesinders @herecomesfran *hugs*,carolinesinders herecomesfran hugs


In [11]:
df = df[['CText', 'oh_label']]
df.rename(columns={'CText':'Text','oh_label':'label'}, inplace = True)
df.label = df.label.astype('int64')

In [12]:
df.label = pd.to_numeric(df.label, errors='coerce')
print(df.dtypes)

Text     object
label     int64
dtype: object


In [13]:
df.Text = str(df['Text']).rstrip()

In [14]:
sorted(df.label.unique())
num_labels = df.label.nunique()
num_labels

2

In [15]:
possible_labels = df.label.unique()
print(possible_labels)

[1 0]


In [16]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [17]:
df['label'] = df.label.replace(label_dict)

In [18]:
from sklearn.model_selection import train_test_split
#target = 'label'

train_df, eval_df = train_test_split(df, test_size=0.2)#,stratify=df.label.values)

In [19]:

from simpletransformers.classification import ClassificationModel

model_args = {
    "num_train_epochs": 1,
    "learning_rate": 1e-5,
    "train_batch_size": 32,
    'overwrite_output_dir':True
}

# Create a ClassificationModel
model = ClassificationModel('roberta', 'distilroberta-base', num_labels=num_labels, use_cuda=False, args=model_args)

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model t

In [20]:
%%time
# Train the model
model.train_model(train_df)

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:379: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10776 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/337 [00:00<?, ?it/s]

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Wall time: 1h 57min 48s


(337, 0.4402910047362749)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [21]:

from sklearn.metrics import f1_score, accuracy_score, classification_report


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)


C:\Users\sofia.zabit\Anaconda3\lib\site-packages\simpletransformers\classification\classification_model.py:900: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2695 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/337 [00:00<?, ?it/s]

C:\Users\sofia.zabit\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [22]:
result

{'mcc': 0.0,
 'tp': 2349,
 'tn': 0,
 'fp': 346,
 'fn': 0,
 'f1': 0.8716141001855287,
 'acc': 0.8716141001855288,
 'eval_loss': 0.3833236592636618}

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [ ]:
result, model_outputs, wrong_predictions

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
_, predictions, true_vals = model.eval_model(eval_df)

In [ ]:
result

In [ ]:
predictions = []
for x in model_outputs:
    predictions.append(np.argmax(x))

#print('f1 score:', f1_score(eval_df['label'], predictions,average='micro'))

#target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(eval_df['label'], predictions,sorted(possible_labels)))#,target_names=sorted(df['label'].unique())))


In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(eval_df['label'], predictions,sorted(possible_labels))

In [ ]:
import seaborn as sns
sns.heatmap(cf, annot=True)